In [ ]:
import tweepy
import twitter

import csv
from datetime import datetime 
import pandas as pd
import numpy as np
import os


CONSUMER_KEY = 'nHSSMHieBZ4glVM9U7QGtI81r'
CONSUMER_SECRET = 'Wsm8z4oxzpq95aajc3El8TMlbmWTD3Vo8EvZ8a6vJta922KJsS'
OAUTH_TOKEN = '721095779335548929-xHLTocT6ClXB2LBkFMZbBRAsLBCjtWt'
OAUTH_TOKEN_SECRET = 'JwYvCraZvEMP1JlopqbRLvOZJI1VfVMQFvEoXAAQ4wksO'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

authT = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
authT.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(authT)

In [ ]:
WORLD_WOE_ID = 1
US_WOE_ID = 23424977

# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.

world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_WOE_ID)

In [ ]:
import json
print (json.dumps(US_WOE_ID, indent=1))

In [ ]:
import urllib.request
import bs4
from bs4 import BeautifulSoup

us_trends_set = list([trend['name'] 
                        for trend in us_trends[0]['trends']])

In [ ]:
def saveFile(df):
    csv = '/Users/LXIN/Desktop/twitter.csv'
    
    if not os.path.isfile(csv):
        df.to_csv(csv, header=True, index=False, encoding='utf-8')
        print("new")
    else:
        df.to_csv(csv, mode='a', header=False, index=False)
        print("exist")

In [ ]:
def saveFrame(data, topics, time, data_len):
    colName = ['Date', 'Topic', 'Comment']
    df = pd.DataFrame(columns = colName)
    timeLst = []
    topicLst = []
    
    for i in range(data_len):
        timeLst.append(time)
        topicLst.append(topics)
    
    df['Date'] = timeLst
    df['Topic'] = topicLst
    df['Comment'] = data
    
    saveFile(df)

In [ ]:
import emoji

# TEST Remove all the emoji 
test = "eak-up juicy details 😩😩"

text = ''.join(c for c in test if c not in emoji.UNICODE_EMOJI)
print(text)

In [ ]:
# XXX: Set this variable to a trending topic, 
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.

import emoji
import time

now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

#print(now)


# Remove the "#" in front of the topic
for i in range(0,30):
    q = us_trends_set[i]
    
    topics = []
    topics.append(q.replace("#",""))
    time = []
    time.append(now)
    data = []
    

    # Total number of tweets extracted: 30
    # Extract the most popular tweets under each trending topic
    tweets = api.search(q=q, result_type='popular', count=50, include_rts = False, tweet_mode = 'extended')
    
    for info in tweets:
        
        text = ''.join(c for c in info.full_text if c not in emoji.UNICODE_EMOJI)    
        data.append(text)

    # If the number of popular tweets is less than 30, extract some latest tweets 
    tweets2 = api.search(q=q , count= (30 - len(data)), include_rts = False, tweet_mode = 'extended')
    for info in tweets2:
        
        text = ''.join(c for c in info.full_text if c not in emoji.UNICODE_EMOJI)  
        
        data.append(text)

    # Store the datetime, trending topic and tweets to file
    saveFrame(data, topics, time, len(data))


print("done")  

In [ ]:
# import pandas as pd
# data = pd.read_csv("/Users/LXIN/Desktop/twitter.csv")
# print(data)